In [1]:
import plotly.express as px
import re
import plotly.io as pio
# pio.renderers.default = "browser"
pio.renderers.default = "notebook_connected"
from collections import OrderedDict
import argparse
from scipy.stats import gmean
import numpy as np
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os

from util import get_list, to_matrix, pattern_map, prune_map, bcolors
from util import spec17_working_set_map, spec17_bench_map

In [2]:
time_str = '20220817094701'
# set__ = [1,2,3,4,5,6,7,8,13,14,15,16,17,18,19,20,21,22,23,24,25]
set__ = [1,2,3,5,6,7,14,15,16,18,19,20,21,23,24]
set_encoding = [spec17_bench_map[n] for n in set__]
sync = [2]
util = [100]
core = 1
base = [0,1,2,3,4,5,6,7,8]
outdir_hist = [f'{time_str}/base{base_}/c{core}_set{set_}_sync{sync_}_u{util_}_{base_+1}/stats.txt' \
               for base_ in base for set_ in set__ for sync_ in sync for util_ in util ]
# id_str = '_0'

bench = [f'base{base_}_set{set_}_s{sync_}_u{util_}' for base_ in base for set_ in set__ for sync_ in sync for util_ in util]
bench_detail = [f'{spec17_bench_map[set_]}' for base_ in base for set_ in set__ for sync_ in sync for util_ in util]
outdir_hist

['20220817094701/base0/c1_set1_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set2_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set3_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set5_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set6_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set7_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set14_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set15_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set16_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set18_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set19_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set20_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set21_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set23_sync2_u100_1/stats.txt',
 '20220817094701/base0/c1_set24_sync2_u100_1/stats.txt',
 '20220817094701/base1/c1_set1_sync2_u100_2/stats.txt',
 '20220817094701/base1/c1_set2_sync2_u100_2/stats.txt',
 '20220817094701/base1/c1_set3_sync2_u1

In [3]:
bench

['base0_set1_s2_u100',
 'base0_set2_s2_u100',
 'base0_set3_s2_u100',
 'base0_set5_s2_u100',
 'base0_set6_s2_u100',
 'base0_set7_s2_u100',
 'base0_set14_s2_u100',
 'base0_set15_s2_u100',
 'base0_set16_s2_u100',
 'base0_set18_s2_u100',
 'base0_set19_s2_u100',
 'base0_set20_s2_u100',
 'base0_set21_s2_u100',
 'base0_set23_s2_u100',
 'base0_set24_s2_u100',
 'base1_set1_s2_u100',
 'base1_set2_s2_u100',
 'base1_set3_s2_u100',
 'base1_set5_s2_u100',
 'base1_set6_s2_u100',
 'base1_set7_s2_u100',
 'base1_set14_s2_u100',
 'base1_set15_s2_u100',
 'base1_set16_s2_u100',
 'base1_set18_s2_u100',
 'base1_set19_s2_u100',
 'base1_set20_s2_u100',
 'base1_set21_s2_u100',
 'base1_set23_s2_u100',
 'base1_set24_s2_u100',
 'base2_set1_s2_u100',
 'base2_set2_s2_u100',
 'base2_set3_s2_u100',
 'base2_set5_s2_u100',
 'base2_set6_s2_u100',
 'base2_set7_s2_u100',
 'base2_set14_s2_u100',
 'base2_set15_s2_u100',
 'base2_set16_s2_u100',
 'base2_set18_s2_u100',
 'base2_set19_s2_u100',
 'base2_set20_s2_u100',
 'base2_se

In [4]:
bench_detail

["['povray']",
 "['cam4']",
 "['nab']",
 "['namd']",
 "['omnetpp']",
 "['lbm']",
 "['xz 1']",
 "['xz 2']",
 "['xz 3']",
 "['deepsjeng']",
 "['leela']",
 "['exchange2']",
 "['x264 1']",
 "['wrf']",
 "['blender']",
 "['povray']",
 "['cam4']",
 "['nab']",
 "['namd']",
 "['omnetpp']",
 "['lbm']",
 "['xz 1']",
 "['xz 2']",
 "['xz 3']",
 "['deepsjeng']",
 "['leela']",
 "['exchange2']",
 "['x264 1']",
 "['wrf']",
 "['blender']",
 "['povray']",
 "['cam4']",
 "['nab']",
 "['namd']",
 "['omnetpp']",
 "['lbm']",
 "['xz 1']",
 "['xz 2']",
 "['xz 3']",
 "['deepsjeng']",
 "['leela']",
 "['exchange2']",
 "['x264 1']",
 "['wrf']",
 "['blender']",
 "['povray']",
 "['cam4']",
 "['nab']",
 "['namd']",
 "['omnetpp']",
 "['lbm']",
 "['xz 1']",
 "['xz 2']",
 "['xz 3']",
 "['deepsjeng']",
 "['leela']",
 "['exchange2']",
 "['x264 1']",
 "['wrf']",
 "['blender']",
 "['povray']",
 "['cam4']",
 "['nab']",
 "['namd']",
 "['omnetpp']",
 "['lbm']",
 "['xz 1']",
 "['xz 2']",
 "['xz 3']",
 "['deepsjeng']",
 "['leela'

### run to check if all required files exist

In [5]:
for path_to_file in outdir_hist:
    if os.path.exists(path_to_file)==False or \
        os.stat(path_to_file).st_size == 0: 
        print(bcolors.FAIL + f'File {path_to_file} is empty or does not exist'+ bcolors.ENDC)

### run to check if any hostSeconds too short to be valid

In [10]:
metric = 'hostseconds'
hs = [get_list(o, metric) for o in outdir_hist]
for i in range(len(hs)):
    s = hs[i]
    assert len(s) == 1
    if s[0] < 10: print(bcolors.WARNING + f'Suspicious {bench[i]} with really short run time of {s[0]}s' + bcolors.ENDC)


Suspicious base0_set24_s2_u100 with really short run time of 6.74s
Suspicious base1_set24_s2_u100 with really short run time of 6.2s
Suspicious base2_set24_s2_u100 with really short run time of 5.13s
Suspicious base3_set24_s2_u100 with really short run time of 4.69s
Suspicious base4_set24_s2_u100 with really short run time of 4.52s
Suspicious base5_set24_s2_u100 with really short run time of 4.46s
Suspicious base6_set24_s2_u100 with really short run time of 4.53s
Suspicious base7_set24_s2_u100 with really short run time of 4.47s
Suspicious base8_set24_s2_u100 with really short run time of 4.4s


In [7]:
mesh_dim = int(np.sqrt(core))
mesh_dim

1

In [29]:
metric = 'l1_t'
l1_miss_allsets = []
for set_ in set__:
    l1_miss_this_set = [get_list(o, metric)[0] for o in outdir_hist if f'set{set_}_' in o]
    print(f'set{set_}')
    print(f'{l1_miss_this_set}')
    l1_miss_allsets.append((set_,l1_miss_this_set))

set1
[23189601, 2240074, 37320, 8072, 7258, 7258, 7258, 7258, 7258]
set2
[346595083, 277386570, 209087339, 159503808, 106229727, 76409230, 76407008, 76407008, 76407008]
set3
[145924725, 119938072, 70958846, 11881308, 8325873, 7753150, 7753164, 7753164, 7753164]
set5
[547101585, 168724449, 41836668, 25347172, 21513057, 19629209, 19629359, 19629359, 19629359]
set6
[157270874, 96998348, 69261137, 53851761, 43399595, 38073253, 38072972, 38072972, 38072972]
set7
[23840008, 22816956, 16560270, 13894931, 13246283, 13028164, 13028165, 13028165, 13028165]
set14
[229961, 185813, 149635, 131372, 114671, 106169, 106167, 106167, 106167]
set15
[353237793, 279614837, 100911259, 73192885, 49013986, 31150697, 31172317, 31172317, 31172317]
set16
[90340, 62192, 46570, 34226, 28688, 25268, 25268, 25268, 25268]
set18
[201725, 64980, 19742, 10440, 9199, 9091, 9091, 9091, 9091]
set19
[18896472, 12961092, 1871249, 313490, 118555, 69060, 69056, 69056, 69056]
set20
[15733749, 1022328, 170044, 62046, 34146, 2534

In [41]:
# x = np.arange(8)
x = [f'{2**n}KB' for n in [5,6,7,8,9,10,11,12,13]]
fig = go.Figure()

for l1_miss_single_set in l1_miss_allsets:
    fig.add_trace(go.Scatter(x=x, y=l1_miss_single_set[1],
                    mode='lines',
                    name=f'{spec17_bench_map[l1_miss_single_set[0]]}',
                    ))
fig.update_yaxes(type="log")
fig.update_layout(title='SPEC working set (access pattern filtered by 32KB L0)',
                   xaxis_title='L1 cache size in KB',
                   yaxis_title='#L1 misses')
fig.show()